<a href="https://colab.research.google.com/github/AWHochman/cds_sp23/blob/main/Optimal_Portfolio_with_fuctions.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Imports
from pandas_datareader import data as web
!pip install yfinance
import yfinance as yf
import pandas as pd
import numpy as np
from datetime import datetime 
from matplotlib import pyplot as plt
plt.style.use('fivethirtyeight')

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
pip install PyPortfolioOpt

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.9/61.9 KB 4.0 MB/s eta 0:00:00


In [ ]:
from pypfopt.efficient_frontier import EfficientFrontier
from pypfopt import risk_models 
from pypfopt import expected_returns
from pypfopt.discrete_allocation import DiscreteAllocation, get_latest_prices

In [ ]:
assets = ['META', 'AMZN', 'AAPL','NFLX','GOOG']
extra_money = 0

# Get portfolio starting date
stockStartDate = '2013-01-01'

# Get portfolio ending date (today)
def today_date():
  today = datetime.today().strftime('%Y-%m-%d')
  return today

In [ ]:
# function to pull stock data from yfinance
# assets is a list of stock tickers (as strings), start is the starting date, and end is the ending date
def get_data(start, end, assets):
  df=pd.DataFrame()
  for stock in assets:
    df[stock]= yf.download(stock, start=start, end=end)['Adj Close']
  return df


In [ ]:
# Portfolio Optimization
# function that finds the optimal weights (and returns them in an array) for each stock. df is a data frame of daily stock prices
def get_weights(df):
  # Calculate the expected returns and the annualized sample covariance matrix of asset returns
  mean = expected_returns.mean_historical_return(df)
  S = risk_models.sample_cov(df)

  # Optimize for max sharpe ratio
  # Sharpe Ratio: A way to describe how much excess return you recieve for the extra amount of volatility you add. 

  ef = EfficientFrontier(mean, S)
  weights = ef.max_sharpe()
  cleaned_weights = ef.clean_weights()

  cleaned_weights_array = [] 
  for key, value in cleaned_weights.items():
   cleaned_weights_array.append(value)
  cleaned_weights_array

  return cleaned_weights_array




In [ ]:
# returns an array with how many shares of each stock should be bought
# df is a data frame of daily stock prices, weights is an array with weights of each stock in df, and money is how much money you can invest
def find_discrete_allocation(df, weights, money):
  latest_prices = get_latest_prices(df)
  spend = np.dot(money,weights) #how much you can spend on each stock 
  num = [] # how much of each stock to buy
  for i in range(len(weights)):
    num.append(np.floor(spend[i]/latest_prices[i]))
  return num


In [ ]:
# finds left over money after investing in the portfolio
# num_stocks is the number of shares of each stock bought
# df must be updated (have today's prices)
def find_money_left_over(num_stocks, df, cash):
   latest_prices = get_latest_prices(df)
   spent = np.dot(num_stocks, latest_prices)
   sum = spent.sum()
   return cash - sum

In [ ]:
#meant to be run after a cycle when rebalancing
# gets total amount from selling portfolio plus our left over money 
def get_profit(num_stocks, df, extra_cash):
  latest_prices = get_latest_prices(df)
  sales = spent = np.dot(num_stocks, latest_prices)
  sum = sales.sum()
  return sum + extra_cash


In [ ]:
money = 100000
df = get_data(stockStartDate, today_date(), assets)
weights = get_weights(df)
a = find_discrete_allocation(df, weights, money)
m = find_money_left_over(a, df, money)


#should be run later (after month)
get_profit(a, df, m) # should be the same as initial money since only used todays data

[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed
[*********************100%***********************]  1 of 1 completed


100000.0